# <center> Контрольна робота  №1 та №2 (ОДЗ)


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

У завданні буде використано набір даних про якість білого вина(репозиторій UCI) 
archive.ics.uci.edu/ml/machine-learning-databases/wine-quality.
Завантажте дані

In [3]:
!mkdir -p data 
!wget -P data -c https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv 

data = pd.read_csv("data/winequality-white.csv", sep=";")
display(data.sample(10))

--2022-11-30 22:07:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264426 (258K) [application/x-httpd-php]
Saving to: ‘data/winequality-white.csv’

winequality-white.c 100%[===================>] 258.23K  --.-KB/s    in 0.06s   

2022-11-30 22:07:12 (4.27 MB/s) - ‘data/winequality-white.csv’ saved [264426/264426]



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4796,6.4,0.105,0.29,1.1,0.035,44.0,140.0,0.99142,3.17,0.55,10.7,7
3682,6.4,0.160,0.44,1.2,0.051,39.0,122.0,0.99058,3.11,0.75,11.3,7
1031,6.2,0.340,0.29,7.6,0.047,45.0,232.0,0.99550,3.35,0.62,10.0,6
3071,6.5,0.360,0.38,10.2,0.028,20.0,82.0,0.99274,3.10,0.43,12.1,7
2323,7.6,0.200,0.34,1.8,0.041,42.0,148.0,0.99335,3.35,0.66,11.1,6
160,6.1,0.320,0.24,1.5,0.036,38.0,124.0,0.98980,3.29,0.42,12.4,7
2967,6.6,0.230,0.30,14.9,0.051,33.0,118.0,0.99835,3.04,0.54,9.0,6
1227,6.9,0.180,0.38,6.5,0.039,20.0,110.0,0.99430,3.10,0.42,10.5,5
289,7.2,0.230,0.39,14.2,0.058,49.0,192.0,0.99790,2.98,0.48,9.0,7
1313,9.2,0.320,0.42,1.3,0.046,14.0,186.0,0.99490,3.08,0.48,9.6,5


In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Відокремте цільову змінну, розділіть навчальну вибірку у відношенні 7:3 (30% - під задишену вибірку, нехай random_state=17) і нормалізуйте дані за допомогою StandartScaler


In [6]:
y = data.quality
data.drop("quality", axis=1, inplace=True)
X_train, X_holdout, y_train, y_holdout = train_test_split(data.iloc[:,:-1], y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Лінійна регресія 


Навчіть просту лінійну модель регресії

In [7]:
linreg =   LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

> **Питання 1 : Які середньоквадратичні помилки лінійної регресії на навчальній і відкоаденій вибірках ?**

In [9]:
print("Mean squared error (train): %.3f" % mean_squared_error(linreg.predict(X_train), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(linreg.predict(X_holdout), y_holdout))

Mean squared error (train): 0.560
Mean squared error (test): 0.601


Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина (врахуйте, що великі за модулем негативні значення коефіцієнтів теж говорять про сильний вплив). Створіть для цього новий невеликий DataFrame.
> **Питання 2 : Яку ознаку лінійна регресія вважає найбільш впливовою на якість вина?**




In [10]:
linreg_coef = pd.DataFrame(data=linreg.coef_, index=X_train.columns, columns=['coef'])
linreg_coef.reindex(linreg_coef.coef.abs().sort_values(ascending=False).index)

,coef
density,-304.623826
volatile acidity,-1.828705
pH,1.250902
sulphates,0.659299
chlorides,0.421804
fixed acidity,0.170026
residual sugar,0.134287
citric acid,0.027948
free sulfur dioxide,0.002071
total sulfur dioxide,0.000510


## Lasso-регресія 

**Навчіть Lasso-регресію з невеликим коефіцієнтом alpha=0,01 (слабка регуляризація). Нехай знову random_state=17.**


In [11]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина. Яка ознака "відпала" першою, тобто найменш важлива для пояснення цільової змінної в моделі Lasso?**

In [12]:
lasso1_coef = pd.DataFrame(data=lasso1.coef_, index=X_train.columns, columns=['coef'])
lasso1_coef.reindex(lasso1_coef.coef.abs().sort_values().index)

,coef
citric acid,0.000000
total sulfur dioxide,0.000000
chlorides,-0.002310
free sulfur dioxide,0.035678
sulphates,0.061058
fixed acidity,0.097925
pH,0.149955
volatile acidity,-0.172850
residual sugar,0.558427
density,-0.764589


**Тепер визначте краще значення alpha в процесі 5-кратної крос-валідації. Використовуйте LassoCV і random_state = 17.**

In [13]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [14]:
lasso_cv.alpha_

1.3200884008314168e-06

Виведіть коефіцієнти "кращого" Lasso в порядку зменшення впливу на якість вина.
> **Питання 3: Яка ознака "занулилася першою" в налаштованій моделі LASSO?**

In [15]:
lasso_cv_coef = pd.DataFrame(data=lasso_cv.coef_, index=X_train.columns, columns=['coef'])
lasso_cv_coef.reindex(lasso_cv_coef.coef.abs().sort_values(ascending=False).index)

,coef
density,-303.693127
volatile acidity,-1.827054
pH,1.247894
sulphates,0.658134
chlorides,0.398770
fixed acidity,0.169219
residual sugar,0.133864
citric acid,0.028389
free sulfur dioxide,0.002083
total sulfur dioxide,0.000497


**Оцініть середньоквадратичну помилку моделі на навчальній і тестовій вибірках.**

> **Питання 4 : Які середньоквадратичні помилки налаштованої LASSO-регресії на навчальній і відкладеній вибірках?**

In [16]:
print("Mean squared error (train): %.3f" % mean_squared_error(lasso_cv.predict(X_train), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lasso_cv.predict(X_holdout), y_holdout))


Mean squared error (train): 0.560
Mean squared error (test): 0.601


## Випадковий ліс

**Навчіть випадковий ліс з параметрами "з коробки", фіксуючи тільки random_state=17.**

In [17]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(random_state=17)

> **Питання 5 : Які середньоквадратичні помилки випадкового лысу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error' і іншими параметрами за замовчуванням) і відкладеній вибірках?**

In [24]:
print("Mean squared error (train): %.3f" % mean_squared_error(forest.predict(X_train), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_holdout), y_holdout))
print("Mean squared error (cv): ", cross_val_score(estimator=forest, X=X_train, y=y_train, scoring='neg_mean_squared_error'))


Mean squared error (train): 0.054
Mean squared error (test): 0.381
Mean squared error (cv):  [-0.45939708 -0.41861662 -0.39786108 -0.3930054  -0.40606161]


**Налаштуйте параметри min_samples_leaf і max_depth за допомогою GridSearchCV і знову перевірте якість моделі на крос-валідації і на відкладеній вибірках.**

In [25]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6, 12))}
locally_best_forest = GridSearchCV(forest, forest_params)
locally_best_forest.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
525 fits failed out of a total of 3150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
525 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py

GridSearchCV(estimator=RandomForestRegressor(random_state=17),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]})

In [27]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 6, 'min_samples_leaf': 1},
 0.48348392193623146)

**Нажал результати GridSearchCV в повному не відтворювані (можуть відрізнятися на різних платформах навіть при фіксованому random_state). Тому навчіть ліс з параметрами max_depth=19, max_features=7, i min_samples_leaf=1 (краще в моэму випадку).**
> **Питання 6 : Які середньоквадратичні помилки налаштованого випадкового лісу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error') і на відкладеній вибірках?**


In [28]:
forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=19, max_features=7, random_state=17)

In [29]:
print("Mean squared error (cv): %.3f" % mean_squared_error(forest.predict(X_train), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_holdout), y_holdout))
print("Mean squared error (cv): ", cross_val_score(estimator=forest, X=X_train, y=y_train, scoring='neg_mean_squared_error'))

Mean squared error (cv): 0.058
Mean squared error (test): 0.382
Mean squared error (cv):  [-0.4548974  -0.40506896 -0.40107211 -0.39072969 -0.39768754]


**Оцініть важливість ознак за допомогою випадкового лісу.**
>**Питання 7 : Яка ознака виявилася найінформативнішою в налаштованій моделі випадкового лісу?**

In [30]:
rf_importance = pd.DataFrame(data=lasso_cv.coef_, index=X_train.columns, columns=['coef'])
rf_importance.reindex(rf_importance.coef.abs().sort_values(ascending=False).index)

,coef
density,-303.693127
volatile acidity,-1.827054
pH,1.247894
sulphates,0.658134
chlorides,0.398770
fixed acidity,0.169219
residual sugar,0.133864
citric acid,0.028389
free sulfur dioxide,0.002083
total sulfur dioxide,0.000497


**Зробіть висновки про якість моделей і оцінки впливу ознак на якість вина за допомогою цих трьох моделей**